## 数据预处理


## 读取数据

In [48]:
import os
import scipy.io
import shutil
import math
import os

#将merge的图片读入dataloader
from torch.utils.data import Dataset, DataLoader

import torch
from torchvision import models
from torch import nn
import torch.nn.functional as F
from refer.process import train_epoch
from refer.models import get_efficientNetB3
from refer.utils import *
from torch.nn import Parameter

from torch.utils.data import random_split


import refer.img_processor as img_processor

import matplotlib.pyplot as plt
from PIL import Image
from refer.Dataset import KinshipDataset




In [ ]:
# non_kin_count = 1
# # 读取 mat 文件
# mat_file_path = 'data\KinFaceW-II\meta_data\\fs_pairs.mat'  # 替换为你的 mat 文件路径
# data = scipy.io.loadmat(mat_file_path)

# # 设定原图像文件夹路径和目标文件夹路径
# original_image_folder = 'data\KinFaceW-II\images\\father-son'  # 替换为你的原图像文件路径
# target_folder = 'data\KinFaceW-II\images\\nonkin'  # 替换为你保存 non-kin 图像的文件夹路径
# if not os.path.exists(target_folder):
#     os.makedirs(target_folder)

# # 解析 mat 文件的结构
# folds = data.get('pairs')


# # 初始化图像对计数器


# # 遍历每个 fold

# for pair in folds:
#     kin_status = pair[1]  # 提取 kin/non-kin 信息
#     image1 = pair[2]
#     image2 = pair[3]
    
#     if kin_status == 0:  # 0 表示 non-kin（不相关的亲缘关系）
#         # 构建新的文件名
#         new_image1_name = f"none_{non_kin_count:03d}_1.jpg"
#         new_image2_name = f"none_{non_kin_count:03d}_2.jpg"
        
#         # 定位源文件路径
#         image1_path = os.path.join(original_image_folder, image1[0])
#         image2_path = os.path.join(original_image_folder, image2[0])

#         # 复制并重命名文件
#         shutil.copy(image1_path, os.path.join(target_folder, new_image1_name))
#         shutil.copy(image2_path, os.path.join(target_folder, new_image2_name))
        
#         # 更新计数器
#         non_kin_count += 1

# print(f"Successfully copied and renamed {non_kin_count-1} non-kin pairs.")

<>:2: SyntaxWarning: invalid escape sequence '\K'
<>:6: SyntaxWarning: invalid escape sequence '\K'
<>:7: SyntaxWarning: invalid escape sequence '\K'
<>:2: SyntaxWarning: invalid escape sequence '\K'
<>:6: SyntaxWarning: invalid escape sequence '\K'
<>:7: SyntaxWarning: invalid escape sequence '\K'
C:\Users\fran\AppData\Local\Temp\ipykernel_34292\2849249646.py:2: SyntaxWarning: invalid escape sequence '\K'
  mat_file_path = 'data\KinFaceW-II\meta_data\\fs_pairs.mat'  # 替换为你的 mat 文件路径
C:\Users\fran\AppData\Local\Temp\ipykernel_34292\2849249646.py:6: SyntaxWarning: invalid escape sequence '\K'
  original_image_folder = 'data\KinFaceW-II\images\\father-son'  # 替换为你的原图像文件路径
C:\Users\fran\AppData\Local\Temp\ipykernel_34292\2849249646.py:7: SyntaxWarning: invalid escape sequence '\K'
  target_folder = 'data\KinFaceW-II\images\\nonkin'  # 替换为你保存 non-kin 图像的文件夹路径


Successfully copied and renamed 1000 non-kin pairs.


In [52]:
class KinshipDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        
        # 定义正样本（亲缘关系）的文件夹路径
        self.kin_folders = ['father-dau', 'father-son', 'mother-dau', 'mother-son','nonkin']
        self.imglabel={"fd":1,"fs":2,"md":3,"ms":4,"nonkin":0,"brothers":5,"sisters":6,"brother-sister":7}
        self.short_relation = {
            'father-dau': 'fd',
            'father-son': 'fs',
            'mother-dau': 'md',
            'mother-son': 'ms',
            'nonkin': 'nonkin'
        }
        # 获取正样本图像对和负样本图像
        self.kin_pairs = self.get_kpairs()
        
        
    def get_kpairs(self):
        """获取亲缘关系图像对 (img1, img2, label)"""
        pairs = []
        for folder in self.kin_folders:
            folder_path = os.path.join(self.root_dir, folder)
            images = sorted(os.listdir(folder_path))  # 假设每个父母-子女的图片名有配对规律
            for i in range(0, len(images)-1, 2):  # 假设成对出现
                if images[i].lower() == 'thumbs.db' or images[i+1].lower() == 'thumbs.db':
                    continue  # 忽略 Thumbs.db 文件
                img1_path = os.path.join(folder_path, images[i])
                img2_path = os.path.join(folder_path, images[i+1])
                # print(folder)
                # print(self.short_relation)
                # print(folder in self.short_relation)
                pairs.append((img1_path, img2_path, self.imglabel[self.short_relation[folder]]))
        return pairs
    
    
    def __len__(self):
        return len(self.kin_pairs) 
    
    def __getitem__(self, idx):
        if idx < len(self.kin_pairs):
            img1_path, img2_path, label = self.kin_pairs[idx]
        else:
            idx -= len(self.kin_pairs)
            img1_path, img2_path, label = self.nonkin_images[idx]
        
        # 加载图片
        img1 = Image.open(img1_path).convert("RGB")
        img2 = Image.open(img2_path).convert("RGB")
        
        # 应用变换
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        
        return (img1, img2), label


### 训练

### model info

In [32]:


class ArcFaceLoss(nn.Module):
    def __init__(self, num_classes, feature_dim, s=30.0, m=0.50, easy_margin=False):
        super(ArcFaceLoss, self).__init__()
        self.num_classes = num_classes
        self.feature_dim = feature_dim
        self.s = s  # Scaling factor
        self.m = m  # Angular margin
        self.easy_margin = easy_margin

        # Initialize weights for each class
        self.weight = nn.Parameter(torch.FloatTensor(num_classes, feature_dim))
        nn.init.xavier_uniform_(self.weight)

        # Precompute cos(m) and sin(m)
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = self.sin_m * m

    def forward(self, features, labels):
        # Normalize features and weights
        features = F.normalize(features, p=2, dim=1)  # Shape: (batch_size, feature_dim)
        weight = F.normalize(self.weight, p=2, dim=1)  # Shape: (num_classes, feature_dim)

        # Compute cosine similarity between features and weights
        cosine = F.linear(features, weight)  # Shape: (batch_size, num_classes)
        sine = torch.sqrt(1.0 - torch.clamp(cosine ** 2, 0, 1))  # Shape: (batch_size, num_classes)
        phi = cosine * self.cos_m - sine * self.sin_m  # Shape: (batch_size, num_classes)

        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        # Create one-hot encoding for labels
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, labels.view(-1, 1).long(), 1.0)

        # Combine phi and cosine based on one-hot labels
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s  # Apply scaling

        loss = F.cross_entropy(output, labels)
        return loss

In [33]:

class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """

    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features)).to(device)
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        # cos(a+b)=cos(a)*cos(b)-size(a)*sin(b)
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output


In [34]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        # 计算欧氏距离
        pos_dist = F.pairwise_distance(anchor, positive, p=2)  # 锚点和正样本的距离
        neg_dist = F.pairwise_distance(anchor, negative, p=2)  # 锚点和负样本的距离

        # Triplet Loss 计算
        loss = torch.relu(pos_dist - neg_dist + self.margin)  # max(0, pos_dist - neg_dist + margin)
        return loss.mean()

In [44]:

from torch.utils.tensorboard import SummaryWriter


# 1. 定义 EfficientNet 模型，并保留特征向量输出，同时加一个线性分类层

class EfficientNetWithHead(nn.Module):
    def __init__(self, num_classes=5):
        super(EfficientNetWithHead, self).__init__()
        self.backbone = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        self.backbone.classifier = nn.Identity()  
        self.feature_dim = None
        self._infer_feature_dim()

        self.fc_1 = nn.Sequential(
            nn.Linear(1280, 1280),
            nn.ReLU(),
            nn.Linear(1280, 512))    
        self.fc_2 = nn.Sequential(
            nn.Linear(1280, 1280),
            nn.ReLU(),
            nn.Linear(1280, 512))
        
        self.fc_3 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes))
    def _infer_feature_dim(self):
        # 利用注册钩子获取特征维度
        def hook(module, input, output):
            self.feature_dim = output.shape[1]
        
        handle = self.backbone.register_forward_hook(hook)
        with torch.no_grad():
            # 随机传入一个输入，触发钩子函数
            dummy_input = torch.randn(1, 3, 224, 224)  
            self.backbone(dummy_input)
        handle.remove()  # 移除钩子以防干扰    
        
    def forward(self, x):
        feature1 = self.backbone(x[0])  # 提取特征向量
        feature2 = self.backbone(x[1])
        t_1 = self.fc_1(feature1)
        t_2 = self.fc_2(feature2)
        
        # 分类器，将特征映射为类别输出
        return self.fc_3(torch.cat((t_1, t_2), dim=1))
    

In [ ]:
batch_size = 2
learning_rate = 1e-05
betas = (0.9, 0.999)
num_workers = 4
weight_decay = 3e-05
pin_memory = True
drop_path_rate = 0.625
gamma = 0.7
num_epochs=20
img_folder = 'data\\KinFaceW-I\\images'
img_folder1 = 'data\\KinFaceW-II\\images'
# Assuming the dataset is already created
dataset = KinshipDataset(img_folder, transform=img_processor.preprocess224_MaxVit)

# Define the lengths for train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model = EfficientNetWithHead(num_classes=5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=betas, weight_decay=weight_decay)

print(train_dataset.__len__(), val_dataset.__len__(), test_dataset.__len__())
print(train_dataset.__getitem__(0)[0][0].shape)


744 106 214
torch.Size([3, 224, 224])


In [ ]:
#TODO 判断是否从上次训练中断，如果是则加载模型

In [50]:
import torch.nn.functional as F

In [58]:


# Initialize SummaryWriter
writer = SummaryWriter()

accuracy = 0.0
for epoch in range(num_epochs):
    model.train()  # 训练模式

    # 前向传播
    for imgs, label in train_loader:
        imgs=(imgs[0].to(device),imgs[1].to(device))
        label = label.float().to(device)
        optimizer.zero_grad()
        output = model(imgs)
        # features, logits = features.to(device), logits.to(device)  # Ensure both are on the same device
        # print(features.shape, logits.shape, label.shape)
        # #打印所在device
        # print(features.device, logits.device, label.device)
        # loss = ArcMarginProduct(in_features=1536, out_features=5)(features, label)
        loss = F.cross_entropy(output, label.long())
        loss.backward()
        optimizer.step()

    # 计算精确度
    
    if epoch % 5 == 0:
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for imgs,label in val_loader:
                imgs=(imgs[0].to(device),imgs[1].to(device))
                label = label.float().to(device)
                output = model(imgs).to(device)
                _, predicted = torch.max(output, 1)
                total += label.size(0)
                correct += (predicted == label).sum().item()
            accuracy = correct / total
        print(f'Epoch: {epoch}, Loss: {loss.item()}, Accuracy: {accuracy}')
    print(f'Epoch: {epoch}, Loss: {loss.item()}, Accuracy: {accuracy}')
    torch.cuda.empty_cache()  # 释放未使用的显存



Epoch: 0, Loss: 0.9253877401351929, Accuracy: 0.5283018867924528
Epoch: 0, Loss: 0.9253877401351929, Accuracy: 0.5283018867924528
Epoch: 1, Loss: 1.0744284391403198, Accuracy: 0.5283018867924528
Epoch: 2, Loss: 1.5979723930358887, Accuracy: 0.5283018867924528
Epoch: 3, Loss: 0.7973589897155762, Accuracy: 0.5283018867924528
Epoch: 4, Loss: 0.9405859708786011, Accuracy: 0.5283018867924528
Epoch: 5, Loss: 1.6454575061798096, Accuracy: 0.6509433962264151
Epoch: 5, Loss: 1.6454575061798096, Accuracy: 0.6509433962264151
Epoch: 6, Loss: 0.687907874584198, Accuracy: 0.6509433962264151
Epoch: 7, Loss: 0.9606838226318359, Accuracy: 0.6509433962264151
Epoch: 8, Loss: 0.7065160274505615, Accuracy: 0.6509433962264151
Epoch: 9, Loss: 0.9817295670509338, Accuracy: 0.6509433962264151
Epoch: 10, Loss: 0.764066219329834, Accuracy: 0.6792452830188679
Epoch: 10, Loss: 0.764066219329834, Accuracy: 0.6792452830188679


KeyboardInterrupt: 

In [33]:
#在测试集上测试
model.eval()
acc_test = 0.0
with torch.no_grad():
    correct = 0
    total = 0
    for img,label in val_loader:
        img,label = img.to(device),label.to(device)
        features, logits = model(img)
        _, predicted = torch.max(logits, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
    acc_test = correct / total

In [32]:
# 5. 保存模型 命名方式：efnet+epoch+batchsize+lr+weightdecay+loss
torch.save(model.state_dict(), f"efnet_{num_epochs}_{batch_size}_{learning_rate}_{loss.item()}.pth")

In [ ]:



_, _, _, start_epoch = load_checkpoint(latest_checkpoint_path(model_dir, "checkpoint_11.pth"), model, optimizer)

train_loss, trian_acc, lr = train_epoch(model, optimizer, None, loss_fn, train_loader, device)
save_checkpoint(model, optimizer, None, 0, model_dir, max_checkpoints = 10)
    

image_files = [f for f in os.listdir(path_to_test_images) if f.endswith('.jpg')]
results = []

model.eval()
with torch.no_grad():
    for i, image_file in enumerate(image_files):
        image_path = os.path.join(path_to_test_images, image_file)
        image = Image.open(image_path).convert('RGB')
        image = transform_val(image).unsqueeze(0).to(device)
        outputs = model(image)
        probabilities = F.softmax(outputs, dim=1)
        prob_class_1 = probabilities[0][1].item()  
        if (i % 100 == 0):
            print(f'Processed {i}/{len(image_files)} images')

        results.append((image_file, prob_class_1))

df = pd.DataFrame(results, columns=['Name', 'Diagnosis'])
df.to_csv(output_csv, index=False)